数据处理工具 torch.utils 和 torchvision

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader

In [ ]:
class TestDataset(Dataset):
    def __init__(self):
        self.Data = np.asarray([
            [1, 2],
            [3, 4],
            [2, 1],
            [3, 4],
            [4, 5],
        ])  # 数据集
        self.Label = np.asarray([0, 1, 0, 1, 2])  # 对应的标签
    
    def __getitem__(self, index):
        txt = torch.from_numpy(self.Data[index])
        label = torch.tensor(self.Label[index])
        return txt, label
    
    def __len__(self):
        return len(self.Data)
    

In [ ]:
t = TestDataset()
print(t[2])
print(len(t))

In [ ]:
# windows 下 num_workers 参数如果设置大于 0 会调用 multiprocessing 模块, 需要在 __name__ == '__main__' 下运行
# https://pytorch.org/docs/stable/data.html#platform-specific-behaviors
loader = DataLoader(t, batch_size=2, shuffle=False, num_workers=0)
for i, train_data in enumerate(loader):
    print('i:', i)
    print('data:', train_data[0])
    print('label:', train_data[1])


In [ ]:
dataiter = iter(loader)
print(next(dataiter))

In [ ]:
pip install tensorboard

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.tensorboard import SummaryWriter
import numpy as np

In [48]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.pool2 = nn.MaxPool2d(2, 2)
        self.bn = nn.BatchNorm2d(20)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2_drop(self.conv2(x))))
        x = self.bn(x)
        x = x.view(-1, 20 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = F.softmax(x, dim=1)
        return x

print(Net())


Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (bn): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)


In [9]:
# 可视化网络
input = torch.rand(32, 1, 28, 28)
model = Net()
with SummaryWriter(log_dir='logs', comment='Net') as w:
    w.add_graph(model, (input, ))

使用以下命令启动 tensorboard

```bash
tensorboard --logdir=logs --port=6006
tensorboard --logdir='D:\code\pytorch_learn\logs' --port=6006
```

In [10]:
# 可视化损失值
input_size = 1
output_size = 1
num_epoches = 60
learning_rate = 0.01

dtype = torch.FloatTensor
writer = SummaryWriter(log_dir='logs',comment='Linear')
np.random.seed(100) 
x_train = np.linspace(-1, 1, 100).reshape(100,1) 
y_train = 3*np.power(x_train, 2) +2+ 0.2*np.random.rand(x_train.size).reshape(100,1) 


model = nn.Linear(input_size, output_size)

criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(num_epoches):
    inputs = torch.from_numpy(x_train).type(dtype)
    targets = torch.from_numpy(y_train).type(dtype)

    output = model(inputs)
    loss = criterion(output, targets)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # 保存loss的数据与epoch数值
    writer.add_scalar('训练损失值', loss, epoch)

In [0]:
# 可视化特征图
import torchvision.utils as utils
# writer = SummaryWriter(log_dir='logs', comment='feature map')
# img_grid = utils.make_grid(x, normalize=True, scale_each=True, nrow=2)

In [8]:
import numpy as np 
import torch
# 数据集
from torchvision.datasets import mnist
# 预处理模块
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
# nn 及 优化器
import torch.nn.functional as F 
import torch.optim as optim
from torch import nn

In [53]:
train_batch_size = 64
test_batch_size = 128
num_epoches = 5
lr = 0.01
momentum = 0.5
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5], [0.5])])
# 下载数据
train_dataset = mnist.MNIST('./data', train=True, transform=transform, download=True)
test_dataset = mnist.MNIST('./data', train=False, transform=transform, download=True)
# dataloader 是个可迭代对象
train_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=True)

# 模型和优化器
model = Net()
model.to(device)  # 注意, 模型也要迁移到 device 上
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

print(device)
print(model)

cuda:0
Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (bn): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)


In [49]:
# 训练模型
print('train_dataset length:', len(train_dataset))
print('train_loader length:', len(train_loader))
len_train_loader = len(train_loader)

for epoch in range(2):  
    running_loss = 0.0
    running_acc = 0.0

    for i, data in enumerate(train_loader, 0):
        # 获取训练数据
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # 权重参数梯度清零
        optimizer.zero_grad()

        # 正向及反向传播
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 累加损失值
        running_loss += loss.item()
        # 计算分类的准确率
        _, pred = outputs.max(1)
        num_correct = (pred == labels).sum().item()
        running_acc += num_correct / inputs.shape[0]
    
    # 显示损失值
    print('[{}] loss: {:.3f}, acc: {:.3f}'.format(
        epoch,
        running_loss / len_train_loader,
        running_acc / len_train_loader,
    ))


train_dataset length: 60000
train_loader length: 938
[0] loss: 1.522, acc: 0.939
[1] loss: 1.525, acc: 0.936


In [15]:
model = Net(28 * 28, 300, 100, 10)
PATH = './model/mnist_model.pth'
model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [54]:
model.eval()
inputs, labels = next(iter(test_loader))
inputs, labels = inputs.to(device), labels.to(device)
x = inputs[0].unsqueeze(0)  # 取出一条数据
print(x.shape)

torch.Size([1, 1, 28, 28])


In [55]:
writer = SummaryWriter(log_dir='logs', comment='feature map')
out = model(x)
print(out)
print(out.shape)
for name, layer in model._modules.items():
    # 为fc层预处理x
    print(x.shape)
    x = x.view(x.size(0), -1) if "fc" in name else x
    print(x.size())
    print(f'{name}')

    x = layer(x)

    # 查看卷积层的特征图
    if  'layer' in name or 'conv' in name:
        x1 = x.transpose(0, 1)  # C，B, H, W  ---> B，C, H, W
        img_grid = utils.make_grid(x1, normalize=True, scale_each=True, nrow=4)  # normalize进行归一化处理
        writer.add_image(f'{name}_feature_maps', img_grid, global_step=0)

tensor([[0.1028, 0.0885, 0.1019, 0.1087, 0.0963, 0.0930, 0.1020, 0.1137, 0.0882,
         0.1049]], device='cuda:0', grad_fn=<SoftmaxBackward>)
torch.Size([1, 10])
torch.Size([1, 1, 28, 28])
torch.Size([1, 1, 28, 28])
conv1
torch.Size([1, 10, 24, 24])
torch.Size([1, 10, 24, 24])
pool1
torch.Size([1, 10, 12, 12])
torch.Size([1, 10, 12, 12])
conv2
torch.Size([1, 20, 8, 8])
torch.Size([1, 20, 8, 8])
conv2_drop
torch.Size([1, 20, 8, 8])
torch.Size([1, 20, 8, 8])
pool2
torch.Size([1, 20, 4, 4])
torch.Size([1, 20, 4, 4])
bn
torch.Size([1, 20, 4, 4])
torch.Size([1, 320])
fc1
torch.Size([1, 50])
torch.Size([1, 50])
fc2
